## Milestone 4: Querying the data

#### Task 1: How many stores does the business have and in which countries?

The Operations team would like to know which countries we currently operate in and which country now has the most stores. Perform a query on the database to get the information, it should return the following information:


In [ ]:
+----------+-----------------+
| country  | total_no_stores |
+----------+-----------------+
| GB       |             265 |
| DE       |             141 |
| US       |              34 |
+----------+-----------------+

In [ ]:
-- Subsequent SQL Query to extract this data

SELECT DISTINCT(country_code) as country,
	COUNT(country_code) as total_no_stores
FROM dim_store_details
GROUP BY country_code
ORDER BY total_no_stores DESC;

#### Task 2: Which locations currently have the most stores?

The business stakeholders would like to know which locations currently have the most stores.

They would like to close some stores before opening more in other locations.

Find out which locations have the most stores currently. The query should return the following:

In [ ]:
+-------------------+-----------------+
|     locality      | total_no_stores |
+-------------------+-----------------+
| Chapletown        |              14 |
| Belper            |              13 |
| Bushley           |              12 |
| Exeter            |              11 |
| High Wycombe      |              10 |
| Arbroath          |              10 |
| Rutherglen        |              10 |
+-------------------+-----------------+

In [ ]:
SELECT DISTINCT locality,
		COUNT(store_code) as total_no_stores
FROM dim_store_details
GROUP BY locality
ORDER BY COUNT(store_code) DESC
LIMIT 7;

#### Task 3: Which months produced the largest amount of sales?

Query the database to find out which months have produced the most sales. The query should return the following information:

In [ ]:
+-------------+-------+
| total_sales | month |
+-------------+-------+
|   673295.68 |     8 |
|   668041.45 |     1 |
|   657335.84 |    10 |
|   650321.43 |     5 |
|   645741.70 |     7 |
|   645463.00 |     3 |
+-------------+-------+

It appears you need to write a joining statement to join three tables to get date, product price & product quantity;
- dim_products (product_code & product_price columns)
- orders_table (product_code, product_quantity & date_uuid columns)
- dim_date_times (month & date_uuid columns)

In [ ]:
SELECT
	ROUND(SUM(orders_table.product_quantity * dim_products.product_price)::numeric,2) AS total_sales,
	dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
GROUP BY dim_date_times.month
ORDER BY SUM(orders_table.product_quantity * dim_products.product_price) DESC
LIMIT 6;

In [ ]:
-- Working out/draft

-- First part

SELECT DISTINCT month
FROM dim_date_times
LIMIT 6;

-- orders_table.product_quantity * dim_products.product_price

-- Total orders per month

SELECT COUNT(orders_table.product_code) AS total_orders,
		dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
GROUP BY dim_date_times.month
ORDER BY COUNT(orders_table.product_code) DESC;

-- You need to multiply orders_table.product_quantity by dim_products.product_price

SELECT (orders_table.product_quantity * dim_products.product_price) AS total_product_sale,
		dim_date_times.month
FROM
	orders_table
INNER JOIN
	dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
INNER JOIN
	dim_products ON dim_products.product_code = orders_table.product_code
GROUP BY dim_date_times.month, orders_table.product_quantity, dim_products.product_price
ORDER BY dim_date_times.month DESC;



#### Task 4: How many sales are coming from online?

The company is looking to increase its online sales.
They want to know how many sales are happening online vs offline.
Calculate how many products were sold and the amount of sales made for online and offline purchases.
You should get the following information:

In [ ]:
+------------------+-------------------------+----------+
| numbers_of_sales | product_quantity_count  | location |
+------------------+-------------------------+----------+
|            26957 |                  107739 | Web      |
|            93166 |                  374047 | Offline  |
+------------------+-------------------------+----------+

You'll need to extract data from the following;
- dim_store_details (store_type & store_code columns)
    - you'll need store_type of Web and everything else grouped as one
- orders_table (product_quantity & card_number columns)
- a way to tally each individual payment completed linked to the above details
    - could that be with dim_card_details (and just card_number column?)

In [ ]:
orders_table will give you sale (product_code) and quantity (product_quantity)

-- Can we find out a way to see number of sales and product quantity per store_type?

To set location as Web or Offline, I believe you will need to create a new column
for the purpose of this task

SELECT
	COUNT(orders_table.product_code) AS number_of_sales,
	SUM(orders_table.product_quantity) AS product_quantity_count,
	dim_store_details.store_type
FROM
	orders_table
INNER JOIN
	dim_store_details ON dim_store_details.store_code = orders_table.store_code
GROUP BY
	dim_store_details.store_type;

-- This will group the sales information by the store type
-- Next you will need to create a new column to define the location Web, Offline